In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

import plotly
plotly.offline.init_notebook_mode(connected=True) 
import plotly.graph_objs as go
import plotly.io as pio

from sklearn import preprocessing

In [2]:
data = pd.read_csv('../sbdc_data_merged.csv')

In [3]:
#data.crosstab('Region','Business Status')

In [4]:
sankey = data.groupby(['Region','center_region'])['center_region'].count().to_frame().rename({'center_region':'count'},axis=1).reset_index()

In [5]:
sankey.head()

,Region,center_region,count
0,Corridor,Baltimore Business Recovery,28
1,Corridor,Corridor,5708
2,Corridor,DUPLICATES,15
3,Corridor,Eastern,34
4,Corridor,Hispanic Business Center,448


In [6]:
region_le = preprocessing.LabelEncoder()
region_le.fit(sankey['Region'])
region_transform = region_le.transform(sankey['Region'])
region_inverse_transform = region_le.inverse_transform(region_transform)
sankey['region_position'] = region_transform

center_le = preprocessing.LabelEncoder()
center_le.fit(sankey['center_region'])
center_transform = center_le.transform(sankey['center_region'])
center_inverse_transform = center_le.inverse_transform(center_transform)
sankey['center_position'] = center_transform

In [7]:
region_label = list(sankey['Region'].unique())
center_label = list(sankey['center_region'].unique())
label = []
label.extend(region_label)
label.extend(center_label)

In [8]:
sankey['center_position'] = sankey['region_position'].max() + sankey['center_position'] + 1

In [9]:
data = dict(
    type='sankey',
    orientation = "h",
    #valueformat = ".4f",
    node = dict(
      pad = 100,
      thickness = 30,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label = label,
      color = "black"
    ),
    link = dict(
      source = sankey['region_position'],
      target = sankey['center_position'],
      value = sankey['count'],
      #label = inverse_transform
      #color = link_color
  ))

title = str("Business Region to SBDC Center")
layout =  dict(
    title = title,
    font = dict(
      size = 20
    ),
    width=1200,
    height=1200,
)

fig = dict(data=[data], layout=layout)
#plotly.offline.iplot(fig, validate=False)
pio.write_image(fig, "{}.png".format(title))
plotly.offline.plot(fig, filename =  "{}.html".format(title), auto_open=False)

'file://C:\\Users\\raymo\\UMD\\Research\\DC2019\\Raymond\\Business Region to SBDC Center.html'